In [13]:
%load_ext autoreload
%autoreload 2
from dl import Variable, Module
import numpy as np
from dl.functions import sum, cross_entropy_loss
from dl.modules import Convolution, Linear, ReLU, Flatten, MaxPool

import torch
import torch.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
def transfer_weights_to_torch(model, torch_model):
    torch_params = [p for p in torch_model.parameters()]
    params = model.parameters()
    
    assert len(torch_params) == len(params), "Parameter count mismatch"

    for torch_p, p in zip(torch_params, params):
        p = torch.tensor(p.data, dtype=torch.float32)
        if p.shape != torch_p.data.shape:
            p = p.T

        torch_p.data.copy_(p)

def compare_grads(model, torch_model):
    torch_params = [p for p in torch_model.parameters()]
    params = model.parameters()
    
    assert len(torch_params) == len(params), "Parameter count mismatch"

    for torch_p, p in zip(torch_params, params):

        # My framework stores W in Linear layer as the transpose so that the output is Y = X @ W + b.
        # This workaround fails for square weights, so come up with a better solution in the future.
        mygrad = p.grad()
        if p.grad().shape != torch_p.grad.shape:
            mygrad = mygrad.T

        print(np.allclose(torch_p.grad.numpy(), mygrad, atol=1e-4, rtol=1e-3))

## Convolution Implementation 

In [15]:
# TODO

# def compare_grads(C_in, C_out, K, stride, padding, N_batch, H, W, verbose=False):
#     # Input
#     X = Variable(np.random.randn(N_batch, C_in, H, W), keep_grad=True)
#     X_torch = torch.tensor(X.data, dtype=torch.float32, requires_grad=True)

#     # Your Conv
#     conv = Convolution(C_in, C_out, K, stride=stride, padding=padding)
#     Y = conv(X)
#     z = sum(Y)
#     z.backward()

#     # PyTorch Conv
#     conv_torch = nn.Conv2d(C_in, C_out, K, stride=stride, padding=padding, bias=False)
#     W_tensor = torch.tensor(conv.W.data, dtype=torch.float32)
#     with torch.no_grad():
#         conv_torch.weight.copy_(W_tensor)
#     Y_torch = conv_torch(X_torch)
#     z_torch = torch.sum(Y_torch)
#     z_torch.backward()

#     grad_torch_W = conv_torch.weight.grad.numpy()
#     grad_mine_W = conv.W.grad()

#     grad_torch_X = X_torch.grad.numpy()
#     grad_mine_X = X.grad()

#     w_close = np.allclose(grad_mine_W, grad_torch_W, atol=1e-4, rtol=1e-3)
#     x_close = np.allclose(grad_mine_X, grad_torch_X, atol=1e-4, rtol=1e-3)

#     if not (w_close and x_close) and verbose:
#         print(f"FAILED: Cin={C_in}, Cout={C_out}, K={K}, stride={stride}, pad={padding}, size=({N_batch},{H},{W})")
#         if not w_close:
#             print("Weight grad mismatch — max diff:", np.max(np.abs(grad_mine_W - grad_torch_W)))
#         if not x_close:
#             print("Input grad mismatch — max diff:", np.max(np.abs(grad_mine_X - grad_torch_X)))

#     return w_close and x_close

# def run_all_tests():
#     paddings = [0, 1, 2, 60]
#     strides = [1, 2, 37, 90]
#     kernels = [1, 3, 5]
#     batch_sizes = [1, 4]
#     image_sizes = [(5, 5), (7, 7), (16, 16), (8, 32)]
#     channels = [(1, 1), (3, 8), (8, 3), (4, 4)]

#     total = 0
#     passed = 0

#     for pad in paddings:
#         for stride in strides:
#             for K in kernels:
#                 for (C_in, C_out) in channels:
#                     for N in batch_sizes:
#                         for (H, W) in image_sizes:
#                             if H + 2 * pad < K or W + 2 * pad < K:
#                                 continue  # Skip invalid kernel size
#                             total += 1
#                             try:
#                                 result = compare_grads(C_in, C_out, K, stride, pad, N, H, W)
#                             except Exception as e:
#                                 print(f"\n❌ Exception for: Cin={C_in}, Cout={C_out}, K={K}, stride={stride}, pad={pad}, size=({N},{H},{W})")
#                                 raise e
#                             if result:
#                                 passed += 1
#                             else:
#                                 print(f"\n❌ Failed for: Cin={C_in}, Cout={C_out}, K={K}, stride={stride}, pad={pad}, size=({N},{H},{W})")
#                                 return
#     print(f"✅ Passed {passed} / {total} tests.")

# run_all_tests()

# def test_edge_cases():
#     assert compare_grads(1, 1, 1, 1, 0, 1, 1, 1)  # Minimal case
#     assert compare_grads(3, 4, 3, 2, 4, 16, 7, 7)  # From your original test
#     assert compare_grads(8, 16, 5, 1, 2, 2, 32, 32)  # Large channels and image
#     assert compare_grads(2, 2, 3, 1, 1, 1, 5, 5)  # Small batch

# test_edge_cases()

## Small CNN

In [17]:
class MiniCNN(Module):

    def __init__(self):
        super().__init__()

        self.conv1 = Convolution(3, 4, 4, 1)
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool(2, 2)
        self.flat1 = Flatten()
        self.lin1 = Linear(4*14*14, 783)
        self.relu2 = ReLU()
        self.lin2 = Linear(783, 100)

    def forward(self, X):

        X = self.conv1(X)
        X = self.relu1(X)
        X = self.maxpool1(X)
        X = self.flat1(X)
        X = self.lin1(X)
        X = self.relu2(X)
        X = self.lin2(X)

        return X
    
class MiniCNNTorch(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 4, 4, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flat1 = nn.Flatten()
        self.lin1 = nn.Linear(4*14*14, 783)
        self.relu2 = nn.ReLU()
        self.lin2 = nn.Linear(783, 100)

    def forward(self, X):
        X = self.conv1(X)
        X = self.relu1(X)
        X = self.maxpool1(X)
        X = self.flat1(X)
        X = self.lin1(X)
        X = self.relu2(X)
        X = self.lin2(X)

        return X
    
N = 16
C_in = 3
H = 32
W = 32

X = Variable(np.random.rand(N, C_in, H, W))
y = Variable(np.random.randint(0,100,size=(N,)))
torch_X = torch.tensor(X.data, dtype=torch.float32)
torch_y = torch.tensor(y.data, dtype=torch.int64)

model = MiniCNN()
torch_model = MiniCNNTorch()

transfer_weights_to_torch(model, torch_model)

Y = model(X)
z = cross_entropy_loss(Y, y)
z.backward()

torch_Y = torch_model(torch_X) 
criterion = nn.CrossEntropyLoss()
torch_z = criterion(torch_Y, torch_y)
torch_z.backward()

compare_grads(model, torch_model)

True
True
True
True
True


In [20]:
[p for p in torch_model.named_parameters()]

[('conv1.weight',
  Parameter containing:
  tensor([[[[ 0.0899,  0.1697,  0.1450,  0.1167],
            [-0.0900,  0.1768,  0.1444,  0.0441],
            [ 0.0633, -0.2430, -0.2675,  0.2555],
            [ 0.0709, -0.0725, -0.0381,  0.0830]],
  
           [[-0.1012, -0.1302,  0.1744,  0.1130],
            [-0.0906, -0.0400,  0.0317, -0.3291],
            [-0.1970,  0.2128, -0.1069,  0.0964],
            [ 0.2199, -0.1555, -0.1341,  0.1829]],
  
           [[ 0.1485,  0.1133, -0.2545,  0.0491],
            [-0.2866,  0.1695, -0.0532, -0.1994],
            [ 0.0366, -0.2024, -0.0646,  0.1049],
            [ 0.1056,  0.2547,  0.0240,  0.2198]]],
  
  
          [[[ 0.2995, -0.1352, -0.3815, -0.2840],
            [-0.2070, -0.1778, -0.1658, -0.2293],
            [-0.0700, -0.0230,  0.0055, -0.2034],
            [-0.2276, -0.1138,  0.1241,  0.2154]],
  
           [[ 0.0719,  0.2426,  0.1248,  0.1459],
            [ 0.0799,  0.2767, -0.0394, -0.3173],
            [ 0.1599, -0.2924,  0.2857